```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CTCS, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *
from SWESimulators import DataAssimilationUtils as dautils

from SWESimulators import WindForcingEnsemble

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

## Running multiple Simulators with different wind direction

While keeping a track on the drifters...

In this case, each particle is a combination of a unique drifter living in a unique model state.

Ideas that might evolve into code:
- It should be optional for a Drifter to have an observation. An additional `.hasObservation()` method needs to be implemented, and included somewhere.


In [ ]:
# Parameters
nx, ny, dx, dy = 50, 50, 5.0, 5.0
dt = 1
f = 0.0
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

In [ ]:
#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


In [ ]:
try:
    if 'ensemble' in globals():
        ensemble.cleanUp()
except TypeError:
    pass
        
numParticles = 30
ensemble = WindForcingEnsemble.WindForcingEnsemble(numParticles, cl_ctx)
ensemble.setGridInfo(nx, ny, dx, dy, dt, boundaryConditions)
ensemble.setParameters(f)

driftersPerSim = 1
print "initializing ensemble with " + str(numParticles) + " particles (simulators)..."
ensemble.init(driftersPerSim)
print "ensemble of " + str(numParticles) + " ocean models, each with " + str(driftersPerSim) + " drifter(s)."
startPos = ensemble.getParticleDrifterPositions()
startObs = ensemble.getTrueStateDrifterPositions()
#print "StartPos:\n", startPos
fig = plt.figure(figsize=(5,5))
plt.plot(startPos[:,0], startPos[:,1], 'mo')
plt.plot(startObs[0], startObs[1], 'co')
plt.ylim([0, ny*dy])
plt.xlim([0, nx*dx])

# initial model integration
#for i in range(10):
for i in range(5):
    print "iteration " + str(i)
    ensemble.step(100)
    pos = ensemble.getParticleDrifterPositions()
    plt.plot(pos[:,0], pos[:,1], 'r.')
    obs = ensemble.getTrueStateDrifterPositions()
    plt.plot(obs[0], obs[1], 'b.')
    #print pos

# Loop doing resampling + model integration
for pf in range(3):
    pos = ensemble.getParticleDrifterPositions()
    plt.plot(pos[:,0], pos[:,1], 'kx')
    obs = ensemble.getTrueStateDrifterPositions()
    plt.plot(obs[0], obs[1], 'kx')
    
    print "Resampling"
    dautils.probabilisticResampling(ensemble, reinitialization_variance=20)
    print "Resampling done"

    pos = ensemble.getParticleDrifterPositions()
    plt.plot(pos[:,0], pos[:,1], 'rs')
    obs = ensemble.getTrueStateDrifterPositions()
    plt.plot(obs[0], obs[1], 'bs')
    #for i in range(10):
    for i in range(5):
        print "iteration " + str(i)
        ensemble.step(20)
        pos = ensemble.getParticleDrifterPositions()
        plt.plot(pos[:,0], pos[:,1], 'r.')
        obs = ensemble.getTrueStateDrifterPositions()
        plt.plot(obs[0], obs[1], 'b.')
        #print pos

pos = ensemble.getParticleDrifterPositions()
plt.plot(pos[:,0], pos[:,1], 'rx')
obs = ensemble.getTrueStateDrifterPositions()
plt.plot(obs[0], obs[1], 'bx')

#distances = ensemble.getDistances()
#print distances
#print "len(distances): ", len(distances)
#print "ensemble.getParticlePositions().shape: ", ensemble.getParticlePositions().shape

ensemble.plotDistanceInfo()

# Animate the DA simulation

In [ ]:
if 'ensemble' in globals():
    ensemble.cleanUp()

numParticles = 30
ensemble = WindForcingEnsemble.WindForcingEnsemble(numParticles, cl_ctx)
ensemble.setGridInfo(nx, ny, dx, dy, dt, boundaryConditions)
ensemble.setParameters(f)

driftersPerSim = 1
ensemble.init(driftersPerSim)
print "ensemble of " + str(numParticles) + " ocean models, each with " + str(driftersPerSim) + " drifter(s)."

fig = plt.figure()
plotter = PlotHelper.EnsembleAnimator(fig, ensemble)

T = 5*10 + 3*10
resampling_points = [49, 59, 69]
sub_t = 10
def animate(i):
    
    if (i>0):
        t = ensemble.step(sub_t)
    else:
        t = 0.0
        
    for rp in resampling_points:
        if i == rp:
            dautils.probabilisticResampling(ensemble, reinitialization_variance=20)

    plotter.plot(ensemble);
    fig.suptitle("Particle Filter with random direction wind = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) 
    
    plotter.plot(ensemble);
    fig.suptitle("Ensemble = " + "{:04.0f}".format(t) + " s", fontsize=18)

anim = animation.FuncAnimation(fig, animate, range(T), interval=300)
plt.close(anim._fig)
anim

## Animate a single ocean state within the ensemble

In [ ]:
if 'ensemble' in globals():
    ensemble.cleanUp()

numParticles = 10
ensemble = WindForcingEnsemble.WindForcingEnsemble(numParticles, cl_ctx)
ensemble.setGridInfo(nx, ny, dx, dy, dt, boundaryConditions)
ensemble.setParameters(f)

driftersPerSim = 1
ensemble.init(driftersPerSim)
print "ensemble of " + str(numParticles) + " ocean models, each with " + str(driftersPerSim) + " drifter(s)."

fig = plt.figure()
eta, hu, hv = ensemble.particles[0].download()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta[2:-2, 2:-2], hu[2:-2, 2:-2], hv[2:-2, 2:-2])
plotter.showDrifters(ensemble.particles[0].drifters)

T = 30
def animate(i):
    if i <= T/2:
        print "iteration " + str(i)
        ensemble.step(30)

        eta, hu, hv = ensemble.particles[0].download()
        plotter.plot(eta[2:-2, 2:-2], hu[2:-2, 2:-2], hv[2:-2, 2:-2])
        plotter.showDrifters(ensemble.particles[0].drifters)
        fig.suptitle("CDKLM16 before Particle filter on wind direction, it = " + str(i), fontsize=18)
        #print pos
    if i == T/2:
        print "Resampling"
        print "position : ", ensemble.particles[0].drifters.getDrifterPositions()
        dautils.probabilisticResampling(ensemble, reinitialization_variance=0)
        print "position : ", ensemble.particles[0].drifters.getDrifterPositions()
        print "Resampling done"

        eta, hu, hv = ensemble.particles[0].download()
        plotter.plot(eta[2:-2, 2:-2], hu[2:-2, 2:-2], hv[2:-2, 2:-2])
        plotter.showDrifters(ensemble.particles[0].drifters)
        fig.suptitle("CDKLM16 at Particle filter on wind direction, it = " + str(i), fontsize=18)
        
    if i > T/2:
        print "iteration " + str(i)
        ensemble.step(30)
        eta, hu, hv = ensemble.particles[0].download()
        plotter.plot(eta[2:-2, 2:-2], hu[2:-2, 2:-2], hv[2:-2, 2:-2])
        plotter.showDrifters(ensemble.particles[0].drifters)
        fig.suptitle("CDKLM16 after Particle filter on wind direction, it = " + str(i), fontsize=18)
        #print pos

anim = animation.FuncAnimation(fig, animate, range(T), interval=300)
plt.close(anim._fig)
anim

In [ ]:
print np.empty(0), len(np.empty(0))

print len(ensemble.particles[:-1])
ensemble.printMaxOceanStates()